In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.20)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.72 sec - Loss 0.098336 - ACC 77.62% - ACC Mean 77.62% - AUC 84.95% - AUC Mean 84.95% - Deter 000
Ite 00050 - 0.10 sec - Loss 0.092353 - ACC 77.72% - ACC Mean 77.80% - AUC 85.59% - AUC Mean 85.30% - Deter 020
Ite 00100 - 0.10 sec - Loss 0.077381 - ACC 77.54% - ACC Mean 78.36% - AUC 84.94% - AUC Mean 85.49% - Deter 044
Ite 00150 - 0.10 sec - Loss 0.056319 - ACC 76.20% - ACC Mean 77.87% - AUC 84.12% - AUC Mean 84.92% - Deter 094
Ite 00200 - 0.12 sec - Loss 0.047174 - ACC 76.85% - ACC Mean 78.01% - AUC 84.64% - AUC Mean 84.72% - Deter 144
Ite 00250 - 0.12 sec - Loss 0.045156 - ACC 78.17% - ACC Mean 78.22% - AUC 84.19% - AUC Mean 85.07% - Deter 194
Ite 00300 - 0.13 sec - Loss 0.044531 - ACC 77.83% - ACC Mean 77.57% - AUC 84.33% - AUC Mean 84.82% - Deter 244
Ite 00350 - 0.13 sec - Loss 0.044161 - ACC 77.02% - ACC Mean 77.64% - AUC 84.86% - AUC Mean 84.82% - Deter 294
Ite 00400 - 0.10 sec - Loss 0.043862 - ACC 77.62% - ACC Mean 77.53% - AUC 84.97% - AUC Mean 84.76% - Deter 344
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 79.96% +- 0.0% - auc: 85.26% +- 0.0%
MEAN - acc: 79.72% +- 0.0% - auc: 87.24% +- 0.0%
KNN - acc: 82.15% +- 0.0% - auc: 85.08% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:44<00:00, 113.32it/s]



GAIN RUN 1/10 - acc: 77.57% +- 6.07% - auc: 84.78% +- 3.38%



100%|██████████| 5000/5000 [00:42<00:00, 116.40it/s]



GAIN RUN 2/10 - acc: 78.28% +- 4.98% - auc: 86.4% +- 3.6%



100%|██████████| 5000/5000 [00:42<00:00, 116.29it/s]


...


GAIN RUN 8/10 - acc: 77.19% +- 2.45% - auc: 83.86% +- 3.54%



100%|██████████| 5000/5000 [00:36<00:00, 136.32it/s]



GAIN RUN 9/10 - acc: 79.4% +- 3.53% - auc: 85.37% +- 2.91%



100%|██████████| 5000/5000 [00:37<00:00, 134.69it/s]



GAIN RUN 10/10 - acc: 81.22% +- 3.54% - auc: 86.93% +- 3.69%

GAIN GLOBAL - acc: 78.64% +- 1.22% - auc: 85.2% +- 1.0%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 79.29% +- 0.0% - auc: 84.34% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 1/10 - acc: 78.8% +- 2.85% - auc: 85.69% +- 3.53%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 2/10 - acc: 82.99% +- 6.19% - auc: 86.37% +- 5.03%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 3/10 - acc: 81.15% +- 1.84% - auc: 85.71% +- 3.21%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 4/10 - acc: 79.62% +- 1.48% - auc: 86.13% +- 2.53%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 5/10 - acc: 78.86% +- 4.62% - auc: 85.76% +- 5.79%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 6/10 - acc: 79.91% +- 1.93% - auc: 84.62% +- 2.25%

Iteration: 0
Iteration: 1
Iteration: 2
Iteratio

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 78.77% +- 0.82% - auc: 83.05% +- 2.04%
SOFTIMPUTE RUN 2/10 - acc: 78.55% +- 2.86% - auc: 83.52% +- 2.82%
SOFTIMPUTE RUN 3/10 - acc: 76.59% +- 3.0% - auc: 83.29% +- 2.56%
SOFTIMPUTE RUN 4/10 - acc: 78.56% +- 3.7% - auc: 85.77% +- 3.22%
SOFTIMPUTE RUN 5/10 - acc: 80.26% +- 3.59% - auc: 86.47% +- 3.05%
SOFTIMPUTE RUN 6/10 - acc: 79.9% +- 3.66% - auc: 85.88% +- 3.77%
SOFTIMPUTE RUN 7/10 - acc: 78.14% +- 2.75% - auc: 84.95% +- 1.79%
SOFTIMPUTE RUN 8/10 - acc: 80.19% +- 4.27% - auc: 84.11% +- 2.76%
SOFTIMPUTE RUN 9/10 - acc: 81.01% +- 2.8% - auc: 86.81% +- 3.27%
SOFTIMPUTE RUN 10/10 - acc: 76.99% +- 3.19% - auc: 82.83% +- 2.37%

SOFTIMPUTE GLOBAL - acc: 78.89% +- 1.37% - auc: 84.67% +- 1.42%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.6442

SINKHORN RUN 1/10 - acc: 78.33% +- 2.41% - auc: 84.02% +- 2.98%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.5952

SINKHORN RUN 2/10 - acc: 80.98% +- 2.89% - auc: 85.88% +- 2.92%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.5867

SINKHORN RUN 3/10 - acc: 80.95% +- 5.53% - auc: 86.94% +- 2.19%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.4959

SINKHORN RUN 4/10 - acc: 79.33% +- 0.32% - auc: 85.05% +- 2.66%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7346

SINKHORN RUN 5/10 - acc: 76.98% +- 3.27% - auc: 82.63% +- 3.35%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.6709

SINKHORN RUN 6/10 - acc: 77.6% +- 3.49% - auc: 83.32% +- 2.23%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.5279

SINKHORN RUN 7/10 - acc: 79.93% +- 5.99% - auc: 85.67% +- 4.09%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.5654

SINKHORN RUN 8/10 - acc: 78.39% +- 6.04% - auc: 85

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 78.72% +- 5.53% - auc: 84.04% +- 3.68%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 79.98% +- 6.27% - auc: 84.26% +- 5.72%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 79.51% +- 4.89% - auc: 87.05% +- 4.13%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 77.45% +- 4.33% - auc: 85.17% +- 3.81%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 78.89% +- 5.73% - auc: 83.85% +- 4.2%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 79.38% +- 5.52% - auc: 85.48% +- 2.84%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 76.99% +- 3.28% - auc: 83.27% +- 2.71%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 79.28% +- 3.86% - auc: 84.19% +- 3.36%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - ac

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is even with MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is even with MEAN
Metric ACC - OUR METHOD is significantly worse than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is even with MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
